In [1]:
import json
from datasets import load_dataset
import numpy as np

dataset_id = "qiaojin/PubMedQA"

dataset = load_dataset(dataset_id, "pqa_labeled",trust_remote_code=True)["train"]
print(dataset, dataset[0])

c:\Users\vohun\anaconda3\envs\py10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 1000/1000 [00:00<00:00, 90860.53 examples/s]

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 1000
}) {'pubid': 21645374, 'question': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?', 'context': {'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.', 'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurri

In [4]:
dataset[0]['context']

{'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.',
  'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoT

In [5]:
def build_context(ex):
  context_str = ''
  ctxs = ex['context']['contexts']
  labels = ex['context']['labels']
  for c,l in zip(ctxs, labels):
    context_str += f"{l}: {c}.\n"
  return {"context_str": context_str}


new_dataset = dataset.map(build_context)
new_dataset[0], new_dataset

Map: 100%|██████████| 1000/1000 [00:00<00:00, 4470.09 examples/s]


({'pubid': 21645374,
  'question': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?',
  'context': {'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.',
    'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD)

In [6]:
from collections import Counter

n_contexts_for_raft = 100
unique_contexts = Counter(new_dataset["context_str"])
count = 0
raft_train_ctx = []
raft_test_ctx = []
for k, v in unique_contexts.items():
  if v > 0 and count < n_contexts_for_raft:
    count += 1
    raft_train_ctx.append(k)
  else:
    raft_test_ctx.append(k)

print(len(raft_test_ctx), len(raft_train_ctx))

#raft_train_ctx = unique_contexts[:100]
#raft_test_ctx = unique_contexts[100:]

900 100


In [8]:
test_dataset = new_dataset.filter(lambda x: x["context_str"] in raft_test_ctx)
test_dataset

Filter: 100%|██████████| 1000/1000 [00:00<00:00, 8000.20 examples/s]


Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision', 'context_str'],
    num_rows: 900
})

In [20]:
from datasets import Dataset

test_dataset_unique_ctx = test_dataset.to_pandas()
test_dataset_unique_ctx = test_dataset_unique_ctx.groupby("context").head(1).reset_index()
test_dataset_unique_ctx = Dataset.from_pandas(test_dataset_unique_ctx)
test_dataset_unique_ctx

Dataset({
    features: ['index', 'question', 'answer', 'relevant_passage_ids', 'id', 'context'],
    num_rows: 4197
})

In [9]:
# Dataset for evaluation raft

# 1070
# 
uploaded_dataset_id = "phatvo/pudmed-raft-test"

test_dataset.push_to_hub(uploaded_dataset_id, split="not_duplicated_test")
new_dataset.push_to_hub(uploaded_dataset_id, split="full_test")
#test_dataset_unique_ctx.map(lambda x: {"context": ' '.join(x["context"].split())}).push_to_hub(uploaded_dataset_id, split="test")
#test_dataset_unique_ctx.push_to_hub(uploaded_dataset_id, split="test_unique_ctx", private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.60s/it]
c:\Users\vohun\anaconda3\envs\py10\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vohun\.cache\huggingface\hub\datasets--phatvo--pudmed-raft-test. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/phatvo/pudmed-raft-test/commit/84d7d5ca9322628ba4c043b5e7f6d2dd7a59ceae', commit_message='Upload dataset', commit_description='', oid='84d7d5ca9322628ba4c043b5e7f6d2dd7a59ceae', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
def save_context_to_file(data:list, filepath):
  with open(filepath, "w") as f:
    _data = [each.replace("\n", "") for each in data]
    _data = [' '.join(each.split()) for each in data]
    f.write("\n".join(_data) + "\n")

save_context_to_file(raft_train_ctx, "../data/context_for_generate_pudmed.txt")

In [11]:
with open("../data/context_for_generate_pudmed.txt", "r") as f:
  print(len(f.readlines()))

100


In [16]:
"""

python3 raft.py \
    --datapath "data/context_for_generate_pudmed.txt" \
    --output "tmp/pubmed-test-100-raft" \
    --distractors 3 \
    --doctype txt \
    --chunk_size 512 \
    --questions 1 \
    --openai_key $OPENAI_KEY \
    --completion_model gpt-4o \
    --splitter breakline --p 0.95

"""

'\n\npython3 raft.py     --datapath "data/context_for_generate_cuad.txt"     --output "tmp/cuad-test-50-raft"     --distractors 3     --doctype txt     --chunk_size 512     --questions 1     --openai_key $OPENAI_KEY     --completion_model gpt-4o     --splitter breakline --p 0.95\n\n'

In [ ]:
# $env:OPENAI_KEY=

In [1]:
from datasets import load_dataset

ds = load_dataset("json", data_files="../tmp/pubmed-test-100-raft.jsonl", )
ds = ds.map(lambda x: {"text": f"{x['instruction']}\nCoT Answer: {x['cot_answer']}"})
ds

c:\Users\vohun\anaconda3\envs\py10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'text'],
        num_rows: 100
    })
})

In [2]:
ds.push_to_hub("phatvo/pubmed-100-train-raft")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.57s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/phatvo/pubmed-100-train-raft/commit/881cbba8e8868cad06b3b5dd2d247cf39d6cfbc2', commit_message='Upload dataset', commit_description='', oid='881cbba8e8868cad06b3b5dd2d247cf39d6cfbc2', pr_url=None, pr_revision=None, pr_num=None)